# Regularization

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

## Loading the pickle file

In [21]:
pickle_file = '/Users/jenkins/sleeba/deep_learning_udacity/not_mnist/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


## Applying L2 regularization in Logistic Regression using `sklearn`

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

**One hot encoded labels are not supported by `sklearn` Logistic Regression model. So the data needs to be modelled as single column labels spanning from 0-9. Train dataset is needed to flattened.**

In [3]:
image_size = 28
num_labels = 10
def reformat(dataset):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    return dataset
train_dataset_log = reformat(train_dataset)
train_labels_log = train_labels
valid_dataset_log = reformat(valid_dataset)
valid_labels_log = valid_labels
test_dataset_log = reformat(test_dataset)
test_labels_log  = test_labels

In [5]:
print("Training data dimension: ",train_dataset_log.shape)
print("Training data labels dimension: ", train_labels_log.shape)
print("Validation data dimension: ",valid_dataset_log.shape)
print("Validation data labels dimension: ", valid_labels_log.shape)
print("Testing data dimension: ",test_dataset_log.shape)
print("Testing data labels dimension: ", test_labels_log.shape)

Training data dimension:  (200000, 784)
Training data labels dimension:  (200000,)
Validation data dimension:  (10000, 784)
Validation data labels dimension:  (10000,)
Testing data dimension:  (10000, 784)
Testing data labels dimension:  (10000,)


## Randomly selecting examples to train, validate and test

In [4]:
idx_train = np.random.choice(np.arange(len(train_dataset_log)), 10000, replace=False)
idx_valid = np.random.choice(np.arange(len(valid_dataset_log)), 3000, replace=False)
idx_test = np.random.choice(np.arange(len(test_dataset_log)), 3000, replace=False)
train_sample_log = train_dataset_log[idx_train]
train_label_sample_log = train_labels_log[idx_train]
valid_sample_log = valid_dataset_log[idx_valid]
valid_label_sample_log = valid_labels_log[idx_valid]
test_sample_log = test_dataset_log[idx_test]
test_label_sample_log = test_labels_log[idx_test]

In [7]:
print("Training data sample dimension: ",train_sample_log.shape)
print("Training data labels sample dimension: ", train_label_sample_log.shape)
print("Validation data sample dimension: ",valid_sample_log.shape)
print("Validation data labels sample dimension: ", valid_label_sample_log.shape)
print("Testing data sample dimension: ",test_sample_log.shape)
print("Testing data labels sample dimension: ", test_label_sample_log.shape)

Training data sample dimension:  (10000, 784)
Training data labels sample dimension:  (10000,)
Validation data sample dimension:  (3000, 784)
Validation data labels sample dimension:  (3000,)
Testing data sample dimension:  (3000, 784)
Testing data labels sample dimension:  (3000,)


## Training the model and checking accuracy

In [21]:
regularization_rate = [1.0, 0.01, 0.001] # this is same as beta = [1, 0.01, 0.001]
for i in regularization_rate:
    model = LogisticRegression(verbose=1, n_jobs=-1, max_iter=800, penalty="l2", C=i)
    model.fit(train_sample_log, train_label_sample_log)
    predicted_train_log = model.predict(train_sample_log)
    predicted_valid_log = model.predict(valid_sample_log)
    predicted_test_log = model.predict(test_sample_log)
    print("\n")
    print("Regularization rate: ", i)
    print ("Prediction accuracy in train set : ", 
           metrics.accuracy_score(train_label_sample_log, predicted_train_log))
    print ("Prediction accuracy in validation set : ", 
           metrics.accuracy_score(valid_label_sample_log, predicted_valid_log))
    print ("Prediction accuracy in test set : ", 
           metrics.accuracy_score(test_label_sample_log, predicted_test_log))
    print("-------------------------------------------")

[LibLinear]

Regularization rate:  1.0
Prediction accuracy in train set :  0.903
Prediction accuracy in validation set :  0.796
Prediction accuracy in test set :  0.872666666667
-------------------------------------------
[LibLinear]

Regularization rate:  0.01
Prediction accuracy in train set :  0.8367
Prediction accuracy in validation set :  0.808333333333
Prediction accuracy in test set :  0.889
-------------------------------------------
[LibLinear]

Regularization rate:  0.001
Prediction accuracy in train set :  0.8142
Prediction accuracy in validation set :  0.800333333333
Prediction accuracy in test set :  0.870666666667
-------------------------------------------


As we see, when we adjusting the regularization weight (`C` is the inverse coefficient of regularization. Refer page [Sklearn-Logistic Regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)) test prediction accuracy is changed  so as validation set. This means, the classifier will perform best with right value of `beta` with out losing it's ability to learn the general trend rather than overfitting to training data. 

## Reformat into shape desirable for `tensorflow`

- Data as a flat matrix
- Labels as float 1-hot encodings

In [5]:
idx_train = np.random.choice(np.arange(len(train_dataset_log)), 10000, replace=False)
idx_valid = np.random.choice(np.arange(len(valid_dataset_log)), 3000, replace=False)
idx_test = np.random.choice(np.arange(len(test_dataset_log)), 3000, replace=False)
train_sample_log = train_dataset_log[idx_train]
train_label_sample_log = train_labels_log[idx_train]
valid_sample_log = valid_dataset_log[idx_valid]
valid_label_sample_log = valid_labels_log[idx_valid]
test_sample_log = test_dataset_log[idx_test]
test_label_sample_log = test_labels_log[idx_test]

In [6]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_sample_log, train_label_sample_log)
valid_dataset, valid_labels = reformat(valid_sample_log, valid_label_sample_log)
test_dataset, test_labels = reformat(test_sample_log, test_label_sample_log)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (10000, 784) (10000, 10)
Validation set (3000, 784) (3000, 10)
Test set (3000, 784) (3000, 10)


In [16]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

## Logistic Regression with `L2` regression using `tensorflow`

### Computational Graph

In [15]:
# This is to expedite the process 
train_subset = 10500
# This is a good beta value to start with

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # They're all constants.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables    
    # They are variables we want to update and optimize.
    beta = tf.Variable(0, dtype=tf.float32)
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases 
    # Original loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels) )
    
    # Loss function using L2 Regularization
    regularizer = tf.nn.l2_loss(weights)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases )
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

### Session

In [16]:
num_steps = 801

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for i in [1, 0.01, 0.001]:
        print("---------------------------------------")
        print("Regularization coefficient (beta): ", i)
        for step in range(num_steps):
            # Run the computations. We tell .run() that we want to run the optimizer,
            # and get the loss value and the training predictions returned as numpy
            # arrays.
            session.run(beta.assign(i))
            _, l, predictions = session.run([optimizer, loss, train_prediction])
            if (step % 100 == 0):
                print('Loss at step {}: {}'.format(step, l))
                print('Training accuracy: {:.1f}'.format(accuracy(predictions, 
                                                         train_labels[:train_subset, :])))
                # Calling .eval() on valid_prediction is basically like calling run(), but
                # just to get that one numpy array. Note that it recomputes all its graph
                # dependencies.
            
                # You don't have to do .eval above because we already ran the session for the
                # train_prediction
                print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), 
                                                           valid_labels)))
        print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels))) 


Initialized
---------------------------------------
Regularization coefficient (beta):  1
Loss at step 0: 3131.05419921875
Training accuracy: 13.4
Validation accuracy: 19.4
Loss at step 100: 1.8850555419921875
Training accuracy: 58.0
Validation accuracy: 56.7
Loss at step 200: 1.9128667116165161
Training accuracy: 56.3
Validation accuracy: 55.6
Loss at step 300: 1.9170057773590088
Training accuracy: 55.2
Validation accuracy: 56.0
Loss at step 400: 1.9099663496017456
Training accuracy: 56.5
Validation accuracy: 56.2
Loss at step 500: 1.9082928895950317
Training accuracy: 55.7
Validation accuracy: 56.0
Loss at step 600: 1.931567668914795
Training accuracy: 56.1
Validation accuracy: 54.7
Loss at step 700: 2.0014266967773438
Training accuracy: 54.3
Validation accuracy: 53.4
Loss at step 800: 1.9024858474731445
Training accuracy: 56.2
Validation accuracy: 56.9
Test accuracy: 62.9
---------------------------------------
Regularization coefficient (beta):  0.01
Loss at step 0: 1.5275335311889

## Three layer neural network with `relu` and `l2` regularization

### Computation graph

In [22]:
batch_size = 128
graph = tf.Graph()
no_hidden_layer_neurons = 1024

with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size)) #10000 x 784
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) # 10000 x 1
    tf_valid_dataset = tf.constant(valid_dataset) 
    tf_test_dataset = tf.constant(test_dataset)
    
    beta = tf.Variable(0, dtype=tf.float32)
    
    # Hidden layer
    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, no_hidden_layer_neurons])) # 784 x 1024
    b1 = tf.Variable(tf.zeros([no_hidden_layer_neurons])) # 1024 x1 
    h1 = tf.add(tf.matmul(tf_train_dataset, w1), b1)
    h1_activation = tf.nn.relu(h1)

    
    # Output layer
    w2 = tf.Variable(tf.truncated_normal([no_hidden_layer_neurons, num_labels])) #1024 x 10
    b2 = tf.Variable(tf.random_normal([num_labels])) #10x1
    
    h2 = tf.add(tf.matmul(h1_activation, w2), b2)
    
    # Converting to softmax for finding prediction accuracy in training data
    train_prediction = tf.nn.softmax(h2)
    
    # Training computation.
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=h2))
    
    # Loss function with L2 Regularization with beta=0.01
    regularizers = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
    loss = tf.reduce_mean(loss + beta * regularizers)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Prediction
    def prediction(data):
        h = tf.add(tf.matmul(data, w1), b1)
        h_relu = tf.nn.relu(h)
        prediction = tf.add(tf.matmul(h_relu, w2), b2)
        return prediction


    valid_prediction = prediction(tf_valid_dataset)
    test_prediction = prediction(tf_test_dataset)

### Session for the graph

In [23]:

num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for i in [1, 0.01, 0.001]:
        print("---------------------------------------")
        print("Regularization coefficient (beta): ", i)
        for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            session.run(beta.assign(i))
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step %d: %f" % (step, l))
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
---------------------------------------
Regularization coefficient (beta):  1
Minibatch loss at step 0: 314964.843750
Minibatch accuracy: 11.7%
Validation accuracy: 19.3%
Minibatch loss at step 500: 2.304509
Minibatch accuracy: 8.6%
Validation accuracy: 9.9%
Minibatch loss at step 1000: 2.306274
Minibatch accuracy: 10.9%
Validation accuracy: 10.6%
Minibatch loss at step 1500: 2.305539
Minibatch accuracy: 5.5%
Validation accuracy: 9.9%
Minibatch loss at step 2000: 2.305341
Minibatch accuracy: 13.3%
Validation accuracy: 9.9%
Minibatch loss at step 2500: 2.313149
Minibatch accuracy: 5.5%
Validation accuracy: 9.9%
Minibatch loss at step 3000: 2.304334
Minibatch accuracy: 4.7%
Validation accuracy: 10.6%
Test accuracy: 10.5%
---------------------------------------
Regularization coefficient (beta):  0.01
Minibatch loss at step 0: 2.305953
Minibatch accuracy: 8.6%
Validation accuracy: 10.6%
Minibatch loss at step 500: 0.691224
Minibatch accuracy: 86.7%
Validation accuracy: 82.2%
M

As we see the results, with suitable value of `beta` the accuracy poised to 91.0 %.

## Demonstration of extreme overfitting using very little data

### Computation graph

In [18]:
batch_size = 100
graph = tf.Graph()
no_hidden_layer_neurons = 1024

with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size)) #10000 x 784
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) # 10000 x 1
    tf_valid_dataset = tf.constant(valid_dataset) 
    tf_test_dataset = tf.constant(test_dataset)
    
    beta = tf.Variable(0, dtype=tf.float32)
    
    # Hidden layer
    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, no_hidden_layer_neurons])) # 784 x 1024
    b1 = tf.Variable(tf.zeros([no_hidden_layer_neurons])) # 1024 x1 
    h1 = tf.add(tf.matmul(tf_train_dataset, w1), b1)
    h1_activation = tf.nn.relu(h1)

    
    # Output layer
    w2 = tf.Variable(tf.truncated_normal([no_hidden_layer_neurons, num_labels])) #1024 x 10
    b2 = tf.Variable(tf.random_normal([num_labels])) #10x1
    
    h2 = tf.add(tf.matmul(h1_activation, w2), b2)
    
    # Converting to softmax for finding prediction accuracy in training data
    train_prediction = tf.nn.softmax(h2)
    
    # Training computation.
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=h2))
    
    # Loss function with L2 Regularization with beta=0.01
#     regularizers = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
#     loss = tf.reduce_mean(loss + beta * regularizers)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Prediction
    def prediction(data):
        h = tf.add(tf.matmul(data, w1), b1)
        h_relu = tf.nn.relu(h)
        prediction = tf.add(tf.matmul(h_relu, w2), b2)
        return prediction


    valid_prediction = prediction(tf_valid_dataset)
    test_prediction = prediction(tf_test_dataset)

### Session

In [20]:

num_steps = 3001

train_dataset_2 = train_dataset[:500, :]
train_labels_2 = train_labels[:500]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
#     session.run(beta.assign(0.01))
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_2[offset:(offset + batch_size), :]
        batch_labels = train_labels_2[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 298.97283935546875
Minibatch accuracy: 12.0
Validation accuracy: 21.1
Minibatch loss at step 500: 1.4662634839623934e-07
Minibatch accuracy: 100.0
Validation accuracy: 75.1
Minibatch loss at step 1000: 8.344616020394824e-08
Minibatch accuracy: 100.0
Validation accuracy: 75.1
Minibatch loss at step 1500: 6.079655179291876e-08
Minibatch accuracy: 100.0
Validation accuracy: 75.1
Minibatch loss at step 2000: 4.7683602844017514e-08
Minibatch accuracy: 100.0
Validation accuracy: 75.1
Minibatch loss at step 2500: 4.0531077161176654e-08
Minibatch accuracy: 100.0
Validation accuracy: 75.1
Minibatch loss at step 3000: 3.576272433747363e-08
Minibatch accuracy: 100.0
Validation accuracy: 75.1
Test accuracy: 83.9


Here we can see the extreme case of overfitting which lead to testing data accuracy to mere 83% but training data accuracy is 100%.

## Dropout and variation in extreme overfitting case

Dropout should only be introduced during training, not evaluation for stochastic stability.